In [1]:
1+1

2

In [7]:
# 检查一下talknet的运行结果
import pickle
new_pred = pickle.load(open('/share2/wangyq/projects/video_dialogue/active_speaker/github_upload/csi/snap/multiturn/talknet/s01e01.pkl', 'rb'))
old_pred = pickle.load(open('/share3/wangyq/resources/C1C/asd/s01e01.pkl', 'rb'))

new_pred = sorted(list(new_pred.items()))
old_pred = sorted(list(old_pred.items()))

In [2]:
# 检查一下noisy和非noisy的标注是什么格式
import pickle
easy = pickle.load(open('/share3/wangyq/resources/C1C/track_sims/s03e01.pkl', 'rb'))
hard = pickle.load(open('/share3/wangyq/resources/C1C/track_sims-hard/s03e01.pkl', 'rb'))

In [4]:
print(len(easy), len(hard))
print(type(easy), type(hard))

105 105
<class 'dict'> <class 'dict'>


In [8]:
import numpy as np
def print_format(data):
    if isinstance(data, np.ndarray):
        return data.shape
    elif isinstance(data, dict):
        return {k: print_format(v) for k, v in data.items()}
    elif isinstance(data, (list, tuple)):
        return [print_format(x) for x in data]
    else:
        return data

print(print_format(list(easy.items())[0]))

['s03e01-002479-002501', [{'frame': (23,), 'bbox': (23, 4), 'name': 'phoebe'}, {'frame': (23,), 'bbox': (23, 4), 'name': 'ross'}, {'frame': (23,), 'bbox': (23, 4), 'name': 'rachel'}]]


In [9]:
print(print_format(list(hard.items())[0]))

['s03e01-002479-002501', [{'frame': (23,), 'bbox': (23, 4), 'name': 'ross'}, {'frame': (23,), 'bbox': (23, 4), 'name': 'rachel'}]]


In [11]:
# 不如将face id也写进去
import os
from tqdm import tqdm
easy_src_base_folder = '/share3/wangyq/resources/C1C/track_sims'
hard_src_base_folder = '/share3/wangyq/resources/C1C/track_sims-hard'
dst_base_folder = '../datasets/face_track_annotations'

for fname in tqdm(os.listdir(easy_src_base_folder)):
    data = pickle.load(open(os.path.join(easy_src_base_folder, fname), 'rb'))
    res = dict()
    for clip_name, clip_data in data.items():
        new_clip_data = list()
        for face_track_i, face_track in enumerate(clip_data):
            face_track['face_track_id'] = face_track_i
            new_clip_data.append(face_track)
        res[clip_name] = new_clip_data
    if fname.startswith('s03'):
        pickle.dump(res, open(os.path.join(dst_base_folder, 'test', fname), 'wb'))
    else:
        pickle.dump(res, open(os.path.join(dst_base_folder, 'train', fname), 'wb'))


100%|██████████| 213/213 [00:06<00:00, 33.21it/s]


In [14]:
for fname in tqdm(os.listdir(os.path.join(dst_base_folder, 'test'))):
    easy_data = pickle.load(open(os.path.join(dst_base_folder, 'test', fname), 'rb'))
    noisy_data = pickle.load(open(os.path.join(hard_src_base_folder, fname), 'rb'))
    assert len(easy_data) == len(noisy_data), f"{len(easy_data)}, {len(noisy_data)}"

    res = dict()
    for clip_name, clip_data in easy_data.items():
        face_names_in_noisy = set([x['name'] for x in noisy_data[clip_name]])
        new_clip_data = [track for track in clip_data if track['name'] in face_names_in_noisy]
        res[clip_name] = new_clip_data
    pickle.dump(res, open(os.path.join(dst_base_folder, 'test-noisy', fname), 'wb'))

100%|██████████| 25/25 [00:00<00:00, 46.51it/s]


In [17]:
# 检查一下结果对不对
test_data = pickle.load(open("/share2/wangyq/projects/video_dialogue/active_speaker/github_upload/datasets/face_track_annotations/test-noisy/s03e01.pkl", "rb"))
print(list(test_data.items())[0])

# 呃 懒得把face track视频发送下来看了 应该没问题吧

('s03e01-002479-002501', [{'frame': array([2479, 2480, 2481, 2482, 2483, 2484, 2485, 2486, 2487, 2488, 2489,
       2490, 2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500,
       2501]), 'bbox': array([[399.51831055,  15.69174862, 505.99285889, 145.64306641],
       [392.62713623,  12.67576027, 500.54089355, 141.45791626],
       [388.3302002 ,   7.92768192, 496.70193481, 136.76965332],
       [386.49960327,   6.23927402, 481.67767334, 117.3915329 ],
       [383.9510498 ,   1.46091342, 482.27142334, 114.604599  ],
       [372.62347412,   2.6733737 , 476.48928833, 119.85094452],
       [366.95462036,   2.58811188, 472.61212158, 119.43971252],
       [360.70095825,   2.04194427, 465.76220703, 119.43115234],
       [354.46881104,   4.12867594, 449.3324585 , 108.83058929],
       [351.40655518,   3.4146719 , 450.05834961, 108.3608551 ],
       [346.16122437,   1.95228338, 439.07025146, 107.87784576],
       [331.03344727,   4.56877279, 434.57171631, 120.182724  ],
       [320.149